In [3]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',include_top=True, input_shape=(224,224,3))


553467096/553467096 [==============================] - 3s 0us/step


In [4]:
for layer in conv_base.layers:
    layer.trainable = False


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
# Set the directories of the training and validation data
train_data_dir = r"/content/drive/MyDrive/charts/charts/train_val"
val_data_dir = r"/content/drive/MyDrive/charts/charts/train_val"

In [41]:
# Add new layers to the pre-trained model
x = conv_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [42]:
model = Model(inputs=conv_base.input, outputs=predictions)

In [43]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

In [44]:
# Set up data augmentation for the training data and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Train the model with the data generators
history = model.fit(
    train_datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=16),
    steps_per_epoch=1600 // 16,
    epochs=10,
    validation_data=val_datagen.flow_from_directory(val_data_dir, target_size=(224, 224), batch_size=16),
    validation_steps=400 // 16)

In [28]:
# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_data_dir =r"/content/drive/MyDrive/charts/charts/test"

In [33]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=16
    )

Found 0 images belonging to 0 classes.


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))


In [ ]:
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)